In [1]:
!pip install pypdf

In [2]:
import os
import pypdf

# =========================================================
# 주인님, 다운로드 받은 폴더 경로를 여기에 넣어주세요!
# =========================================================
TARGET_DIR = os.path.abspath("downloads_final_completed") 
# =========================================================

def check_pdf_validity(folder_path):
    print(f"ㅇㅅㅇ: 검사를 시작합니다! 폴더: {folder_path}\n")
    
    if not os.path.exists(folder_path):
        print("❌ 폴더를 찾을 수 없어요!")
        return

    files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]
    total = len(files)
    
    corrupted_files = [] # 깨진 파일 목록
    valid_count = 0

    print(f"총 {total}개의 PDF 파일을 발견했습니다. 검사 중...\n")

    for i, filename in enumerate(files, 1):
        file_path = os.path.join(folder_path, filename)
        is_valid = True
        error_msg = ""

        # 1차 검사: 파일 헤더 확인 (HTML인지 PDF인지)
        try:
            with open(file_path, 'rb') as f:
                header = f.read(4)
                if header != b'%PDF':
                    is_valid = False
                    error_msg = "헤더 오류 (HTML 파일일 가능성 높음)"
        except Exception as e:
            is_valid = False
            error_msg = f"파일 읽기 실패: {e}"

        # 2차 검사: 라이브러리로 실제 페이지 열어보기
        if is_valid:
            try:
                reader = pypdf.PdfReader(file_path)
                # 페이지가 0개거나, 읽을 때 에러가 나면 깨진 파일
                if len(reader.pages) == 0:
                    is_valid = False
                    error_msg = "페이지 없음 (빈 파일)"
                else:
                    # 첫 페이지 텍스트 추출 시도 (가벼운 읽기 테스트)
                    _ = reader.pages[0].extract_text()
            except Exception as e:
                is_valid = False
                error_msg = f"PDF 구조 손상 ({e})"

        # 결과 처리
        if is_valid:
            valid_count += 1
            # 너무 많으니 50개 단위로만 로그 출력
            if i % 50 == 0:
                print(f"[{i}/{total}] 진행 중... (현재까지 정상)")
        else:
            print(f"❌ [손상됨] {filename}")
            print(f"   └ 사유: {error_msg}")
            corrupted_files.append(filename)

    print("-" * 50)
    print(f"검사 완료! 🎉")
    print(f"✅ 정상 파일: {valid_count}개")
    print(f"❌ 손상 파일: {len(corrupted_files)}개")
    
    if corrupted_files:
        print("\n[손상된 파일 목록 - 다시 받아야 해요 ㅠㅠ]")
        for f in corrupted_files:
            print(f"- {f}")
    else:
        print("\n와! 모든 파일이 완벽합니다! 고생하셨어요 주인님! ₍ ˶•⤙•˶ ₎")

if __name__ == "__main__":
    check_pdf_validity(TARGET_DIR)

ㅇㅅㅇ: 검사를 시작합니다! 폴더: c:\Users\Playdata\workplace\final\practice\downloads_final_completed

총 327개의 PDF 파일을 발견했습니다. 검사 중...

[50/327] 진행 중... (현재까지 정상)
[100/327] 진행 중... (현재까지 정상)
[150/327] 진행 중... (현재까지 정상)
[200/327] 진행 중... (현재까지 정상)
[250/327] 진행 중... (현재까지 정상)
[300/327] 진행 중... (현재까지 정상)
--------------------------------------------------
검사 완료! 🎉
✅ 정상 파일: 327개
❌ 손상 파일: 0개

와! 모든 파일이 완벽합니다! 고생하셨어요 주인님! ₍ ˶•⤙•˶ ₎


In [3]:
import os
import re
import time
import random
import requests
import ssl
import urllib3
from requests.adapters import HTTPAdapter
from urllib3.poolmanager import PoolManager
from urllib3.util.ssl_ import create_urllib3_context
from datetime import date
from urllib.parse import urlparse, urljoin

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====== 설정 ======
LIST_URL = "https://www.hyundaicard.com/cpu/ug/CPUUG2001_08.hc"
# ★ 주인님, 아까 성공한 파일들이 들어있는 폴더 경로를 그대로 쓰세요!
OUT_DIR = os.path.abspath("downloads_final_completed") 
MAX_ITEMS = None
RETRY_PER_ITEM = 3
# ==================

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class LegacySSLAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        ctx = create_urllib3_context()
        ctx.options |= 0x4
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        self.poolmanager = PoolManager(
            num_pools=connections,
            maxsize=maxsize,
            block=block,
            ssl_context=ctx
        )

def sanitize_filename(s: str) -> str:
    s = re.sub(r"<[^>]+>", "", s)
    s = re.sub(r"[\\/:*?\"<>|]", "_", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s[:100]

def parse_kr_date(s: str) -> date:
    try:
        parts = [p.strip() for p in s.split(".") if p.strip()]
        if len(parts) >= 3:
            return date(int(parts[0]), int(parts[1]), int(parts[2]))
    except:
        pass
    return date.today()

def get_base_url(current_url: str) -> str:
    p = urlparse(current_url)
    return f"{p.scheme}://{p.netloc}"

def close_popup_hyundai(driver):
    try:
        driver.execute_script("if(window.popup) window.popup.close('popDownload', 'btnDownload');")
        time.sleep(0.5) 
    except: pass
    try:
        driver.execute_script("var btn=document.querySelector('.layer_close a'); if(btn) btn.click();")
    except: pass

def extract_latest_pdf_info(driver, base_url):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#popup_content tr")))
    rows = driver.find_elements(By.CSS_SELECTOR, "#popup_content tr")
    
    candidates = []
    for row in rows:
        try:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 2: continue
            
            d = parse_kr_date(tds[0].text.strip())
            link = row.find_element(By.CSS_SELECTOR, "a.ico_download_link")
            href = link.get_attribute("href")
            if not href: continue
            
            full_url = href if href.startswith("http") else urljoin(base_url, href)
            candidates.append((d, full_url))
        except: continue

    if not candidates:
        raise RuntimeError("팝업 목록 없음")

    return max(candidates, key=lambda x: x[0])

def download_file_with_requests(driver, url, save_path):
    session = requests.Session()
    adapter = LegacySSLAdapter()
    session.mount("https://", adapter)
    
    for cookie in driver.get_cookies():
        session.cookies.set(cookie['name'], cookie['value'])
    
    headers = {
        "User-Agent": driver.execute_script("return navigator.userAgent;"),
        "Referer": LIST_URL
    }

    try:
        response = session.get(url, headers=headers, stream=True, timeout=60, verify=False)
        if response.status_code != 200:
            raise RuntimeError(f"HTTP {response.status_code}")
        
        content_type = response.headers.get("Content-Type", "").lower()
        if "pdf" not in content_type and "octet-stream" not in content_type:
            raise RuntimeError(f"Invalid Content-Type: {content_type}")

        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                
        if os.path.getsize(save_path) < 1000:
            os.remove(save_path)
            raise RuntimeError("파일 용량 미달")
            
    except Exception as e:
        if os.path.exists(save_path):
            os.remove(save_path)
        raise e

    return True

def main():
    if not os.path.exists(OUT_DIR):
        os.makedirs(OUT_DIR)

    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    options.page_load_strategy = 'normal'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)

    print(f"ㅇㅅㅇ: 누락된 파일 채우기 모드 시작! (Skip 로직 개선됨)")

    try:
        driver.get(LIST_URL)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
        
        btns = driver.find_elements(By.CSS_SELECTOR, "a.ico_download")
        items = []
        for b in btns:
            items.append({
                "sqno": b.get_attribute("sqno"),
                "idxdetail": b.get_attribute("idxdetail"),
                "title": b.get_attribute("webblbdtitl") or b.text or "제목없음"
            })
            
        print(f"총 {len(items)}개 항목 스캔 중...")
        
        success_cnt, skip_cnt, fail_cnt = 0, 0, 0
        
        for i, item in enumerate(items, 1):
            sqno = item['sqno']
            idx = item['idxdetail']
            raw_title = item['title']
            safe_title = sanitize_filename(raw_title)
            
            # =================================================================
            # ★ 핵심 수정 포인트: startswith 체크 시 뒤에 '_'를 붙임!
            # 이렇게 하면 '네이버 현대카드'는 '네이버 현대카드 Edition2'와 매칭되지 않음!
            # =================================================================
            existing = [f for f in os.listdir(OUT_DIR) if f.startswith(safe_title + "_")]
            
            if existing:
                # 이미 진짜배기 파일이 있으면 패스 (로그 지저분하니까 출력 생략하거나 간단히)
                # print(f"[{i}] {safe_title} -> 이미 있음 (Skip)")
                skip_cnt += 1
                continue

            print(f"\n[{i}/{len(items)}] {safe_title} -> 📥 다운로드 시작 (누락분 발견!)")

            for attempt in range(1, RETRY_PER_ITEM + 1):
                try:
                    if not driver.current_url.startswith(LIST_URL):
                        driver.get(LIST_URL)
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
                    
                    try:
                        wait.until(EC.invisibility_of_element_located((By.ID, "popDownload")))
                    except:
                        close_popup_hyundai(driver)

                    time.sleep(1.0) 

                    driver.execute_script(f"getDownloadList({sqno}, '{idx}', '{raw_title}');")
                    
                    latest_date, pdf_url = extract_latest_pdf_info(driver, get_base_url(driver.current_url))
                    
                    close_popup_hyundai(driver)
                    
                    final_name = f"{safe_title}_{latest_date.strftime('%Y%m%d')}.pdf"
                    final_path = os.path.join(OUT_DIR, final_name)
                    
                    download_file_with_requests(driver, pdf_url, final_path)
                    
                    print(f"   ✅ 성공! -> {final_name}")
                    success_cnt += 1
                    break 

                except Exception as e:
                    print(f"   ⚠️ 재시도 {attempt}: {e}")
                    
                    try:
                        close_popup_hyundai(driver)
                        driver.refresh()
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ico_download")))
                        time.sleep(2) 
                    except:
                        pass
                    
                    if attempt == RETRY_PER_ITEM:
                        print(f"   ❌ 최종 실패: {safe_title}")
                        fail_cnt += 1

    except Exception as e:
        print(f"치명적 오류: {e}")
        
    finally:
        driver.quit()
        print(f"\n==== 작업 완료 ====")
        print(f"새로 받음: {success_cnt}")
        print(f"이미 있어서 넘김: {skip_cnt}")
        print(f"실패: {fail_cnt}")

if __name__ == "__main__":
    main()

ㅇㅅㅇ: 누락된 파일 채우기 모드 시작! (Skip 로직 개선됨)
총 384개 항목 스캔 중...

[23/384] SC제일은행-the Red -> 📥 다운로드 시작 (누락분 발견!)
   ✅ 성공! -> SC제일은행-the Red_20250203.pdf

[44/384] SC제일은행-현대카드M -> 📥 다운로드 시작 (누락분 발견!)
   ✅ 성공! -> SC제일은행-현대카드M_20260102.pdf

[61/384] 현대카드 Summit -> 📥 다운로드 시작 (누락분 발견!)
   ✅ 성공! -> 현대카드 Summit_20260102.pdf

[67/384] 현대카드M -> 📥 다운로드 시작 (누락분 발견!)
   ✅ 성공! -> 현대카드M_20260102.pdf

[69/384] 현대카드ZERO Edition3(할인형) -> 📥 다운로드 시작 (누락분 발견!)
   ⚠️ 재시도 1: 팝업 목록 없음
   ⚠️ 재시도 2: Invalid Content-Type: text/html; charset=utf-8
   ⚠️ 재시도 3: Invalid Content-Type: text/html; charset=utf-8
   ❌ 최종 실패: 현대카드ZERO Edition3(할인형)

[79/384] SC제일은행-the Red Stripe -> 📥 다운로드 시작 (누락분 발견!)
   ✅ 성공! -> SC제일은행-the Red Stripe_20250203.pdf

[84/384] the Red Stripe -> 📥 다운로드 시작 (누락분 발견!)
   ✅ 성공! -> the Red Stripe_20260102.pdf

[86/384] SC제일은행-현대카드 MY COMPANY -> 📥 다운로드 시작 (누락분 발견!)
   ✅ 성공! -> SC제일은행-현대카드 MY COMPANY_20260102.pdf

[95/384] SC제일은행-현대카드MY BUSINESS ZERO Food&Drink -> 📥 다운로드 시작 (누락분 발견!)
   ⚠️ 재시도 1: Invalid C

In [4]:
import os
from collections import defaultdict

# =========================================================
# 주인님, 파일이 들어있는 폴더 경로를 넣어주세요!
# =========================================================
TARGET_DIR = os.path.abspath("downloads_final_completed") 
# =========================================================

def find_intruder(folder_path):
    print(f"ㅇㅅㅇ: 범인 색출을 시작합니다! (경로: {folder_path})\n")
    
    if not os.path.exists(folder_path):
        print("❌ 폴더가 없어요!")
        return

    files = os.listdir(folder_path)
    pdf_files = [f for f in files if f.lower().endswith('.pdf')]
    non_pdf_files = [f for f in files if not f.lower().endswith('.pdf')]
    
    print(f"📂 전체 파일: {len(files)}개")
    print(f"📄 PDF 파일: {len(pdf_files)}개")
    
    # 1. PDF가 아닌 파일 찾기 (범인 1순위)
    if non_pdf_files:
        print(f"\n[검거 1] PDF가 아닌 녀석들이 있어요! ({len(non_pdf_files)}개)")
        for f in non_pdf_files:
            print(f"  👉 {f}")
    else:
        print("\n[통과] 모든 파일이 PDF입니다.")

    # 2. 이름이 비슷한 중복 파일 찾기 (범인 2순위)
    # 예: "카드명_날짜.pdf" 와 "카드명_날짜_2.pdf"
    print("\n[검거 2] 이름이 겹치는 중복 파일 검사 중...")
    
    # 파일명에서 '_202...' 날짜 부분을 뺀 '카드 이름'만 추출해서 그룹화
    name_groups = defaultdict(list)
    for f in pdf_files:
        # 날짜(_202X...) 앞부분만 잘라서 키로 사용
        # (단순하게 뒤에서부터 _ 숫자를 찾는 방식 등 사용)
        base_name = f.rsplit('_', 1)[0] 
        name_groups[base_name].append(f)

    duplicate_found = False
    for base, file_list in name_groups.items():
        if len(file_list) > 1:
            # 같은 카드 이름으로 파일이 2개 이상인 경우
            print(f"\n🚨 의심스러운 중복 발견! (기준명: {base})")
            for f in file_list:
                print(f"  - {f}")
            duplicate_found = True

    if not duplicate_found:
        print("\n[통과] 이름이 겹치는 파일은 없어 보입니다.")

    # 3. 파일 크기가 0인 파일 찾기 (다운로드 실패 잔해)
    empty_files = [f for f in pdf_files if os.path.getsize(os.path.join(folder_path, f)) == 0]
    if empty_files:
        print(f"\n[검거 3] 내용이 텅 빈(0KB) 파일 발견! ({len(empty_files)}개)")
        for f in empty_files:
            print(f"  👉 {f}")

    print("\n" + "="*30)
    print("수사 종료! 범인을 찾으셨나요? ₍ ˶•⤙•˶ ₎")

if __name__ == "__main__":
    find_intruder(TARGET_DIR)

ㅇㅅㅇ: 범인 색출을 시작합니다! (경로: c:\Users\Playdata\workplace\final\practice\downloads_final_completed)

📂 전체 파일: 385개
📄 PDF 파일: 385개

[통과] 모든 파일이 PDF입니다.

[검거 2] 이름이 겹치는 중복 파일 검사 중...

[통과] 이름이 겹치는 파일은 없어 보입니다.

수사 종료! 범인을 찾으셨나요? ₍ ˶•⤙•˶ ₎
